# OLAP Queries & Analysis

# 1. Imports

In [ ]:
import os, sqlite3
import pandas as pd
import matplotlib.pyplot as plt

# 2. Paths

In [ ]:
OLAP_DIR = os.getcwd()
DB_PATH = os.path.join(os.path.dirname(OLAP_DIR), "etl", "retail_dw.db")
DB_PATH

# 3. Run OLAP queries

In [ ]:
con = sqlite3.connect(DB_PATH)

rollup = pd.read_sql_query("""
SELECT c.Country, t.Year, t.Quarter, ROUND(SUM(s.TotalSales),2) AS TotalSales
FROM SalesFact s
JOIN CustomerDim c ON s.CustomerKey = c.CustomerKey
JOIN TimeDim t     ON s.TimeKey     = t.TimeKey
GROUP BY c.Country, t.Year, t.Quarter
ORDER BY t.Year, t.Quarter, c.Country;
""", con)

drilldown_uk = pd.read_sql_query("""
SELECT t.Year, t.Month, t.MonthName, ROUND(SUM(s.TotalSales),2) AS TotalSales
FROM SalesFact s
JOIN CustomerDim c ON s.CustomerKey = c.CustomerKey
JOIN TimeDim t     ON s.TimeKey     = t.TimeKey
WHERE c.Country = 'United Kingdom'
GROUP BY t.Year, t.Month, t.MonthName
ORDER BY t.Year, t.Month;
""", con)

slice_electronics = pd.read_sql_query("""
SELECT ROUND(SUM(s.TotalSales),2) AS ElectronicsSales
FROM SalesFact s
JOIN ProductDim p ON s.ProductKey = p.ProductKey
WHERE p.Category = 'Electronics';
""", con)

con.close()

rollup.head(), drilldown_uk.head(), slice_electronics


# 4. Plot sales by country & export results

In [ ]:
# Bar chart: total sales by country (sum across quarters)
country_totals = rollup.groupby("Country")["TotalSales"].sum().sort_values(ascending=False)

plt.figure(figsize=(8,5))
country_totals.plot(kind="bar")   # default matplotlib styling
plt.title("Total Sales by Country (Last Year)")
plt.ylabel("Sales")
plt.xlabel("Country")
plt.tight_layout()
plt.savefig(os.path.join(OLAP_DIR, "sales_by_country.png"), dpi=150)
plt.show()

# Save data artifacts
rollup.to_csv(os.path.join(OLAP_DIR, "rollup_country_quarter.csv"), index=False)
drilldown_uk.to_csv(os.path.join(OLAP_DIR, "drilldown_uk_monthly.csv"), index=False)
slice_electronics.to_csv(os.path.join(OLAP_DIR, "slice_electronics.csv"), index=False)
